# The Core Concepts

<details>
<summary><strong>📚 Click to expand Core Concepts section</strong></summary>

<br>

> **Note:** This section provides a quick mental model of how PhariaKernel components work together. Detailed implementation steps follow in the next notebooks.

## Quick Overview

PhariaKernel lets you deploy AI components as serverless functions. Here's the workflow:

1. **Write** a Skill (Python function with AI logic)
2. **Build** it to WebAssembly 
3. **Publish** to a registry
4. **Configure** which Skills are available
5. **Invoke** via REST API

<div align="center">
  <img src="./images/skill_flow.png" width="600" alt="PhariaKernel Deployment Workflow">
  <p><em>Complete deployment workflow from development to production</em></p>
</div>

---

## Concept 1: The PhariaKernel Instance

**What:** A serverless runtime specifically designed for AI workloads - think AWS Lambda, but optimized for AI.

**Why it matters:** Handles all infrastructure complexity (scaling, queuing, load balancing) so you focus only on AI logic.

**To connect:** You need two environment variables from your PhariaKernel operator:
```bash
PHARIA_KERNEL_ADDRESS=<your-instance-url>
PHARIA_AI_TOKEN=<your-auth-token>
```

---

## Concept 2: Skills

**What:** Python functions that encapsulate AI logic with defined input/output schemas using Pydantic models.

**Why it matters:** Skills are stateless, strongly-typed, and testable - perfect for production AI systems.

<div align="center">
  <img src="./images/skill_structure.png" width="400" alt="Skill Structure">
  <p><em>Anatomy of a Skill: Input Schema → Business Logic → Output Schema</em></p>
</div>

**Example:**
```python
from pharia_skill import ChatParams, Csi, Message, skill
from pydantic import BaseModel

class Input(BaseModel):
    topic: str

class Output(BaseModel):
    haiku: str

@skill
def create_haiku(csi: Csi, input: Input) -> Output:
    response = csi.chat(
        "llama-3.1-8b-instruct",
        [Message.user(f"Write a haiku about {input.topic}")],
        ChatParams(max_tokens=64)
    )
    return Output(haiku=response.message.content)
```

---

## Concept 3: Building Skills

**What:** Compile your Python Skill to WebAssembly for secure, portable execution.

**Why it matters:** WASM provides isolation, consistent performance, and language independence.

<div align="center">
  <img src="./images/skill_build.png" width="500" alt="Build Process">
  <p><em>Build process transforms Python code into portable WebAssembly</em></p>
</div>

**Command:**
```bash
uv run pharia-skill build my_skill
```
This creates `my_skill.wasm` ready for deployment.

---

## Concept 4: Skill Registry

**What:** A container registry (like Docker Hub) but for WASM components.

**Why it matters:** Central storage for versioned Skills that PhariaKernel can load on-demand.

<div align="center">
  <img src="./images/skill_registry.png" width="450" alt="Skill Registry">
  <p><em>OCI-compliant registries store and version your Skills</em></p>
</div>

**Setup:** Configure registry credentials (get from your DevOps team):
```bash
export SKILL_REGISTRY=registry.example.com      # Registry URL
export SKILL_REPOSITORY=team/skills             # Your namespace
export SKILL_REGISTRY_USER=<username>           # Your username  
export SKILL_REGISTRY_TOKEN=<token>             # Access token
```

**Publish:**
```bash
uv run pharia-skill publish my_skill.wasm --name my_skill
```

---

## Concept 5: Namespace Configuration

**What:** A TOML file that declares which Skills should be available in your namespace.

**Why it matters:** Controls what Skills are loaded and accessible via API.

<div align="center">
  <img src="./images/skill_namespace.png" width="500" alt="Namespace Configuration">
  <p><em>GitOps workflow: Configuration changes trigger automatic Skill deployment</em></p>
</div>

**Example `namespace.toml`:**
```toml
skills = [
    { name = "my_skill", tag = "latest" },
    { name = "another_skill", tag = "v1.0.0" }
]
```

**Update process:** 
1. Edit namespace.toml in your Git repo
2. Create pull request
3. Merge → PhariaKernel auto-loads new Skills

---

## Concept 6: Invoking Skills

**What:** Call your deployed Skills via REST API.

**Example:**
```bash
curl 'https://pharia-kernel.domain/v1/skills/{namespace}/{skill_name}/run' \
  --header "Authorization: Bearer $PHARIA_AI_TOKEN" \
  --header 'Content-Type: application/json' \
  --data '{"topic": "mountains"}'
```

**Response:**
```json
{
  "haiku": "Peaks touch the sky high\nSnow-capped giants stand in silence\nClouds dance at their feet"
}
```

---

## 🎯 Key Takeaway

PhariaKernel transforms your Python AI functions into production-ready, scalable services with just a few commands. The complexity is hidden - you write Python, PhariaKernel handles the rest.

> **Need Details?** Contact your PhariaKernel operator for environment-specific setup. They'll provide connection details, registry access, and namespace configuration.

</details>

---

## 3 Configure and publish the Skill

The Skill component contains the core logic of your application, including document retrieval and answer generation. Unlike the UI and service components, the Skill is designed to be executed by the PhariaKernel platform rather than running directly on your local machine. However, we need to set up the development environment locally.

### 3.1 Configure the skill

First, configure the environment variables for the OCI registry (<a href="https://oras.land/docs/#what-are-oci-registries">What are OCI registries</a>), which are used to publish the skill. This refers to the .env file in the project root:

```bash
SKILL_REGISTRY=registry.your-organisation.com
SKILL_REPOSITORY=your-pharia-skills-repository
SKILL_REGISTRY_USER=your-registry-username
SKILL_REGISTRY_TOKEN=your-registry-password
```

Second, in `qa.py` (located in `<Application-name>/skills`), update the `NAMESPACE`, `COLLECTION`, and `INDEX` values:

```python
NAMESPACE = "Studio"
COLLECTION = "papers"
INDEX = "asym-64"
```

Replace these with the values you configured in [**1. Setup collections and indexes with PhariaSearch**](../1.%20Enable%20searching%20documents/1.%20Setup%20collections%20and%20indexes%20with%20PhariaSearch.ipynb).

In a ```new terminal window``` navigate to the skill directory and set up the environment:

In [ ]:
cd <your-app-name>/skills

Load the environment variables and install dependencies.

**Choose the command for YOUR operating system:**

#### For Linux/macOS:
```bash
set -a && source ../.env
```

#### For Windows PowerShell:
```powershell
Get-Content "../.env" | ForEach-Object {
    if ($_ -match '^\s*([^#][^=]+?)\s*=\s*(.*)\s*$') {
        $key = $matches[1].Trim()
        $value = $matches[2].Trim('"')  # Optional: remove quotes
        [System.Environment]::SetEnvironmentVariable($key, $value, "Process")
    }
}
```

In [ ]:
uv sync --dev

### 3.2 Publish the Skill

We need now to add the Skill to the Kernel runtime in order to make it available for production use.

In your terminal, ensure you are in the Skill folder (for example, enter `cd <your-app-name>/skill`). First, you build the Skill. This process packages your code and dependencies into a container that can be deployed on PhariaKernel:

In [ ]:
cd <your-app-name>/skill
uv run pharia-skill build qa

If you choose to publish immediately (by typing 'y'), you can skip the next publish command. Otherwise, use the publish command shown below:

In [ ]:
# If you didn't publish during build (chose 'n'), you can publish separately:
uv run pharia-skill publish qa.wasm --name qa-rag-tutorial --tag latest

The last step is to configure PhariaKernel to recognise and serve your Skill. Update your ```namespace.toml``` in your OCI registry (ask your PhariaAI Operator for the location of this file) by adding the new Skill entry.

<div align="center">
  <img src="./images/build_skill_actual.png" width="650" alt="Actual Build Process Output"><br>
  <em>Example output when building a Skill</em>
</div>

<br><br>

<div align="center">
  <img src="./images/publish_skill_actual.png" width="650" alt="Actual Publish Process Output"><br>
  <em>Example output when publishing a Skill to the registry</em>
</div>